# CMSC 320 - Final Tutorial - UMD Subreddit

In [12]:
import requests

import sqlite3
from sqlite3 import Error

import os
from os import path

import pandas as pd

from nltk import FreqDist
from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('wordnet')


In [13]:
def make_get_request(URL, headers=None, given_params=None):
    """Sends a GET request to the given URL.
    
    Parameters
    ----------
    URL : str
        The url to send a GET request
    given_params : dictionary, optional
        A dictionary of any additional parameters (default is None)
        
        
    Returns
    -------
    dictionary
        A dictionary containing the JSON response
    """
    
    SUCCESS = 200
    response = requests.get(URL, headers=headers, params=given_params)
    
    if (response.status_code == SUCCESS):
        return response
    else:
        return {};

# Create a database functionality

In [14]:
def create_connection(db_file):
    conn = None
    print(db_file)
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [15]:
def create_table(conn, create_table_sql, params=()):
    try:
        c = conn.cursor()
        c.execute(create_table_sql, params)
    except Error as e:
        print(e)

# Scrape and Parse Faculty

## Attempted to use umd.io, but it appears to be rather glitchy

In [16]:
"""
umd_professor_url = "https://api.umd.io/v1/professors";
page = 1
professorNames = set() 
params = {'departments': 'CMSC', 'page': page}

json = make_get_request(umd_professor_url, params)
professorNames.update([professor['name'] for professor in json])

while json is not None:
    page = page + 1
    params['page'] = page
    
    json = make_get_request(umd_professor_url, params)
    if json:
        professorNames.update([professor['name'] for professor in json])
        
    print(professorNames)
        
# print(professorNames)
"""
print()

In [17]:
faculty_names_filepath = os.path.abspath('./data/db/faculty_names.db')
should_scrape_names = not path.exists(faculty_names_filepath)

## Scrape faculty from UMD faculty page

In [18]:
from bs4 import BeautifulSoup

# Headers for the request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Allow-Methods": "GET"
}

if should_scrape_names:
    faculty_url = "https://academiccatalog.umd.edu/undergraduate/administrators-officials-faculty/"
    response = make_get_request(faculty_url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')

## Parse each listing into a name, type, and college

In [19]:
def split_name(name):
    # Name structure: <last name>, <first name> (<middle name/initial>)
    split = name.split(',')
    last_name = split[0]
    
    first_name = split[1].split()[0]
    
    return (first_name, last_name)

#### Ignoring those who don't fall into the colleges listed below

In [20]:
colleges = ['AGNR', 'ARCH', 'ARHU', 'BSOS', 'BMGT', 'CMNS', 'EDUC', 'ENGR', 'JOUR', 'INFO', 'SPHL', 'PLCY']

# Description format: Type, SCHOOL-DEPARTMENT (repeated); Degree, University, Year; (repeated)
def create_faculty(name, description, faculty_dict):
    (first_name, last_name) = split_name(name.strip())
    
    job_type = description.split(',')[0]
    college_dept = description.split(',')[1]
    
    # Not currently using, but may switch to it
    faculty_college = college_dept.split('-')[0].strip()
    
    prof_colleges = []
    for college in colleges:
        if college in college_dept:
            prof_colleges.append(college)
    
    if prof_colleges:
        faculty = {'first_name': first_name, 'last_name': last_name, 'colleges': prof_colleges, 'type': job_type}

        if name not in faculty_dict:
            key = last_name + ", " + first_name
            faculty_dict[key] = faculty

In [21]:
def read_names_from_db():
    conn = create_connection(faculty_names_filepath)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM names")
    
    all_faculty = {}
    
    for faculty in cursor.fetchall():
        person = {'first_name': faculty[1], 'last_name': faculty[2], 'colleges': faculty[5], 'type': faculty[4]}
        key = person["last_name"] + ", " + person["first_name"]
        all_faculty[key] = person
        
    return all_faculty

In [22]:
if should_scrape_names:
    faculty_blocks = soup.find_all("p", class_="faculty-item")

    # Keeping track of only CS and ENGR professors, but this is arbitrary
    CMNS_profs = set()
    ENGR_profs = set()
    all_faculty = {}

    for block in faculty_blocks:

        # Names are stored within the <strong/> tag
        name = block.strong.string 

        # content structure: space,  name, space, <br/>, description
        contents = block.contents
        description = contents[4]

        create_faculty(name, description, all_faculty)

        if "CMNS" in description:
            CMNS_profs.add(name)

        elif "ENGR" in description:
            ENGR_profs.add(name)
else:
    all_faculty = read_names_from_db()
        

G:\UMD\FALL_2020\CMSC320\final-tutorial\data\db\faculty_names.db


In [23]:
print(all_faculty["Teli, Mohammad"])

{'first_name': 'Mohammad', 'last_name': 'Teli', 'colleges': 'ENGR', 'type': 'Lecturer'}


## Create a database holding faculty information

In [24]:
# Using only the first mentioned college for now, could switch later? Also only considering the colleges that were listed - others are ignored

def create_names_table(conn):
    names_table = """ CREATE TABLE IF NOT EXISTS names (
                        id integer PRIMARY KEY,
                        first_name text NOT NULL,
                        last_name text NOT NULL,
                        nick_name text,
                        type text NOT NULL,
                        college text NOT NULL,
                        UNIQUE (first_name, last_name, type)
                ) """
    create_table(conn, names_table)

In [25]:
def insert_faculty_name(conn, faculty):
    sql = """ INSERT OR IGNORE INTO names(first_name,last_name,type,college)
                VALUES(?,?,?,?)"""
    
    cursor = conn.cursor()
    cursor.execute(sql, faculty)
    conn.commit()
    
    return cursor.lastrowid

In [26]:
faculty_names_path = os.path.abspath('.')
faculty_names_path = faculty_names_path + "/data/db/faculty_names.db"
faculty_names_db = create_connection(faculty_names_path)

create_names_table(faculty_names_db)
                                      

for faculty_name in all_faculty:
    faculty = all_faculty[faculty_name]
    
    if not faculty["colleges"]:
        college = None
    else:
        college = faculty["colleges"][0]
        
    params = (faculty["first_name"], faculty["last_name"], faculty["type"], college)
    
    insert_faculty_name(faculty_names_db, params)
    
faculty_names_db.close()

G:\UMD\FALL_2020\CMSC320\final-tutorial/data/db/faculty_names.db


## Create a database per professor for comments and submissions mentioning them

### Should only be done when a mention is found

In [27]:
def normalize_name(name):
    return ''.join(char for char in name if char.isalnum())

In [28]:
def create_professor_tables(conn):
    submissions_table = """ CREATE TABLE IF NOT EXISTS submissions (
                            id integer PRIMARY KEY,
                            sub_id text NOT NULL,
                            title text NOT NULL,
                            selftext text NOT NULL,
                            score integer NOT NULL,
                            full_link text NOT NULL,
                            created_time integer NOT NULL,
                            UNIQUE(sub_id)
                        )"""
    
    comments_table = """ CREATE TABLE IF NOT EXISTS comments (
                            id integer PRIMARY KEY,
                            sub_id text NOT NULL,
                            comment_id text NOT NULL,
                            body text NOT NULL,
                            score integer NOT NULL,
                            created_time integer NOT NULL,
                            UNIQUE(comment_id, sub_id)
                    )"""
    
    create_table(conn, submissions_table)
    create_table(conn, comments_table)

In [29]:
def create_professor_db(filename):
    directory = os.path.abspath('.')
    filepath = directory + "\data\db\professors\\" + filename
    
    professor_db = create_connection(filepath)
    
    create_professor_tables(professor_db)
    
    return professor_db

In [30]:
def insert_submission(conn, values):
    insert_sql = """INSERT OR IGNORE INTO submissions (
                        sub_id, title, selftext, score, full_link, created_time
                    ) VALUES (?,?,?,?,?,?)
                 """
    cursor = conn.cursor()
    cursor.execute(insert_sql, values)
    conn.commit()
    
    return cursor.lastrowid

In [31]:
def insert_all_submissions(conn, df):
    for idx, sub in df.iterrows():
        insert_submission(conn, (sub['id'], sub['title'], sub['selftext'], sub['score'], sub['full_link'], sub['created_utc']))

In [32]:
# This will create a database per professor immediately, no matter what. But we probably don't want that
"""
directory = os.path.abspath('.')
directory = directory + "\data\db\professors"

for professor in all_faculty:
    prof_name = split_name(professor)
    prof_name = (normalize_name(prof_name[0]), normalize_name(prof_name[1]))
    filename = prof_name[0] + "_" + prof_name[1] + ".db" # first_last
    
    create_professor_db(filename)
    
"""
print()

## Requesting Data from the UMD Subreddit with PushShift API and BigQuery

In [33]:
import praw
from psaw import PushshiftAPI

api = PushshiftAPI()

In [34]:
def form_college_query(college):
    return "({lower}|{upper})".format(lower=college.lower(), upper=college)

In [36]:
def make_college_filename(college):
    return normalize_name(college + ".db")

In [39]:
umd_subreddit = 'umd'
min_count = 50

sub_limit = 100
sub_filter = ['title', 'selftext', 'score', 'full_link', 'id', 'created_utc']

def query_submissions_mentioning_college(college):
    query = form_college_query(college)
    print(query)
    
    submissions = api.search_submissions(q=query, subreddit=umd_subreddit, filter=sub_filter, limit=sub_limit)
    df = pd.DataFrame([thing.d_ for thing in submissions])
    
    if len(df.index) == min_count:
        return df
        """
        filename = make_professor_filename(professor)
        prof_db = create_professor_db(filename)
        
        df['selftext'] = df['selftext'].apply(lambda x: x.replace("\n", " ").replace("\t", " "))
        insert_all_submissions(prof_db, df)
        prof_db.close()
        """
        
    return df 

In [42]:
cmns_df = query_submissions_mentioning_college('CMNS')

print(cmns_df.info())

engr_df = query_submissions_mentioning_college('ENGR')
print(engr_df.info())

(cmns|CMNS)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   created_utc  97 non-null     int64  
 1   full_link    97 non-null     object 
 2   id           97 non-null     object 
 3   score        97 non-null     int64  
 4   selftext     97 non-null     object 
 5   title        97 non-null     object 
 6   created      97 non-null     float64
dtypes: float64(1), int64(2), object(4)
memory usage: 5.4+ KB
None
(engr|ENGR)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   created_utc  16 non-null     int64  
 1   full_link    16 non-null     object 
 2   id           16 non-null     object 
 3   score        16 non-null     int64  
 4   selftext     16 non-null     object 
 5   title        16 non-null    

### Requesting data with the PushShift API

In [255]:
comment_limit = 10
comment_filter = ['body', 'score', 'created_utc', 'id', 'link_id']

unmentioned_names = []

query_all_professors = False

if query_all_professors:
    for professor_key in all_faculty:
        professor = all_faculty[professor_key]
        
        query_submissions_mentioning_professor(professor)

### Requesting stored data with Google's BigQuery and PushShift

In [256]:
from google.cloud import bigquery

# Analyze Word Frequency

In [257]:
import re
import itertools
import collections

teli_df = query_submissions_mentioning_professor(all_faculty["Teli, Mohammad"])
all_text = teli_df['selftext'].tolist() + teli_df['title'].tolist()
clean_text = [re.sub(r'[^A-Za-z0-9 ]+', '', x).lower() for x in all_text]

words_per_message = [text.split() for text in clean_text]

all_words = list(itertools.chain(*words_per_message))

count_all_words = collections.Counter(all_words)
#print(count_all_words)

In [258]:
default_stopwords = set(stopwords.words('english'))

all_words = [word for word in all_words if word not in default_stopwords]

# There is ConditionalFreqDist in NLTK that'll categorize words as well
freqDist = nltk.FreqDist(all_words)

#for word, freq in freqDist.most_common(50):
    # print("{}: {}".format(word, freq))
   

In [163]:
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()
lemmas = [wnl.lemmatize(word) if word != "cs" else word for word in all_words if word not in default_stopwords]

freqDistLemmas = nltk.FreqDist(lemmas)

for lemma, freq in freqDistLemmas.most_common(50):
    print("{}: {}".format(lemma, freq))

teli: 102
class: 90
351: 50
semester: 49
kruskal: 47
im: 42
take: 40
exam: 40
cmsc351: 31
get: 30
taking: 27
would: 27
cs: 24
like: 23
know: 22
grade: 21
section: 20
also: 19
cmsc: 19
question: 18
good: 18
lecture: 18
two: 17
course: 17
homework: 17
next: 16
pretty: 16
lot: 16
one: 16
time: 16
ta: 16
teaching: 15
really: 15
much: 15
dont: 15
fall: 15
people: 15
got: 14
professor: 14
point: 14
want: 13
project: 13
426: 13
451: 13
waitlist: 13
hard: 13
sure: 12
telis: 12
make: 12
student: 12


#### We could also use bi or tri-gram analysis with NLTK

Large query of multiple CS professors and X professors. Equal amounts of both

How often are negative/postive terms used with these departments? But where's the machine learning....?

### More NLP-like

Large query of all CMSC/CMNS professors + terms of 'CS', 'CMSC', and 'CMNS'

Large query of all X department professors + X terms

Equal amounts of both. ~500 of each? 